In [133]:
import numpy as np
import math
import pandas as pd
import networkx as nx
import random
import collections
import matplotlib.pyplot as plt

# Trabajo final para evaluación de Métodos Numéricos de la Mestría de Ciencia de Datos (equipo 3)
* Jorge García (202945)
* Sandra España (203200)
* Aline Pérez (203096)
* Marco Antonio Ramos (142244)

Este trabajo busca implementar un problema de optimización de distancia recorrida en el metro de la CDMX.

# Creación de la matriz de adyacencias entre las estaciones del metro

Se obtuvo la base de datos de longitud entre estaciones del metro de la CDMX (long_interestacion) y la longitud de la estación (long_estacion). De la misma manera, se creó una lista con todas las estaciones del metro. Los datos originales se pueden consultar en la siguiente [liga](https://metro.cdmx.gob.mx/longitud-de-estacion)

In [145]:
metro1=pd.read_excel('/Users/1thaliaespana1/Documents/Semestre I/Métodos Numéricos/final/distancia_metro.xlsx')
np.set_printoptions(precision=2)
nodos1=list(dict.fromkeys(pd.concat([metro1["Estacion1"],metro1["Estacion2"]])))
N=len(nodos1)

Se creó la matriz de adyacencias, en la cual los nodos son las estaciones y los valores son las distancias en metros

In [146]:
df1=np.zeros((N,N))
adyacencia1=pd.DataFrame(df1)
adyacencia1.columns=adyacencia1.index=nodos1

for i in nodos1:
    for j in nodos1:
        adyacencia1[i][j]=metro1[(metro1["Estacion1"]==i)&(metro1["Estacion2"]==j)]["Long_interestacion"]

adyacencia1=adyacencia1.fillna(0)
adyacencia1=np.transpose(adyacencia1)+adyacencia1
print(adyacencia1)

                      Pantitlan  Zaragoza  GomezFarias  BoulevardPuertoAereo  \
Pantitlan                   0.0    1320.0          0.0                   0.0   
Zaragoza                 1320.0       0.0        762.0                   0.0   
GomezFarias                 0.0     762.0          0.0                 611.0   
BoulevardPuertoAereo        0.0       0.0        611.0                   0.0   
Balbuena                    0.0       0.0          0.0                 595.0   
...                         ...       ...          ...                   ...   
MartinCarrera               0.0       0.0          0.0                   0.0   
BarrancadelMuerto           0.0       0.0          0.0                   0.0   
Constitucionde1917          0.0       0.0          0.0                   0.0   
LaPaz                       0.0       0.0          0.0                   0.0   
Buenavista                  0.0       0.0          0.0                   0.0   

                      Balbuena  Moctezu

Verificamos que no haya columnas que no tengan distancias en ninguna de sus entradas

In [149]:
print("máxima distancia por estación\n")
print("\n",np.max(adyacencia1))
print("\nverificamos que no haya un máximo igual a cero (es decir, que todas las estaciones tengan al menos una adyacencia)\n")
print("\n",np.sort(np.max(adyacencia1)))

máxima distancia por estación


 Pantitlan               1644.0
Zaragoza                1320.0
GomezFarias              762.0
BoulevardPuertoAereo     611.0
Balbuena                 703.0
                         ...  
MartinCarrera           1141.0
BarrancadelMuerto       1476.0
Constitucionde1917      1137.0
LaPaz                   1956.0
Buenavista               521.0
Length: 164, dtype: float64

verificamos que no haya un máximo igual a cero (es decir, que todas las estaciones tengan al menos una adyacencia)


 [ 445.  456.  516.  521.  564.  574.  587.  602.  611.  611.  611.  620.
  634.  637.  645.  653.  657.  657.  659.  665.  665.  698.  702.  703.
  703.  709.  709.  715.  725.  725.  732.  745.  745.  750.  755.  757.
  761.  762.  774.  788.  789.  793.  793.  794.  817.  817.  843.  860.
  884.  908.  908.  908.  910.  910.  924.  924.  942.  950.  959.  968.
  969.  969.  973.  993.  993. 1033. 1033. 1042. 1059. 1060. 1060. 1062.
 1067. 1072. 1084. 1106. 1110. 1110. 1111

/Users/1thaliaespana1/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)
/Users/1thaliaespana1/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


# Creación del grafo de la matriz de adyacencias